In [1]:
import pandas as pd
import numpy as np
import os
import sys

import matplotlib as plt

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### For reproducibility, please make sure PUIDATA environmental variable is properly set up

In [2]:
PUIdata = os.getenv('PUIDATA')
print(PUIdata)

/home/cusp/uc288/PUIdata


In [3]:
path = PUIdata + '/ADSProject'

if (not os.path.isdir(path)):
    if (os.system('mkdir ' + path) == 0):
        print("OK, directory in place")
else:
    print("OK, directory in place")

OK, directory in place


### Downloading Manhattan 2015 Sales
Access all other files here: http://www1.nyc.gov/site/finance/taxes/property-annualized-sales-update.page

http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2015/2015_manhattan.xls

In [4]:
if (not os.path.isfile(path + '/2015_manhattan.xls')):
    print("File not found")
    print("Download file for reading...")
    os.system('curl -O http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2015/2015_manhattan.xls')
    os.system('mv 2015_manhattan.xls ' + path)

if os.path.isfile(path + '/2015_manhattan.xls'):
    print("File in place, read file")
    mn2015 = pd.read_excel(path + '/2015_manhattan.xls', skiprows=4)

File in place, read file


In [5]:
mn2015.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,400,19,,A4,526 EAST 5TH STREET,,...,1,0,1,1883,5200,1900,1,A4,6500000,2015-03-18
1,1,ALPHABET CITY,03 THREE FAMILY DWELLINGS,1,376,24,,C0,264 EAST 7TH STREET,,...,3,0,3,2059,3696,1900,1,C0,3775000,2015-10-22
2,1,ALPHABET CITY,03 THREE FAMILY DWELLINGS,1,377,66,,C0,243 EAST 7TH STREET,,...,3,0,3,2381,3084,1899,1,C0,2900000,2015-06-24
3,1,ALPHABET CITY,04 TAX CLASS 1 CONDOS,1C,399,1101,,R6,238 EAST 4TH STREET,-,...,1,0,1,0,0,1955,1,R6,6995000,2015-06-15
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,373,16,,C1,326 EAST 4TH STREET,,...,10,0,10,2204,8625,1899,2,C1,20000000,2015-11-17


In [12]:
if (not os.path.isfile(path + '/2015_bronx.xls')):
    print("File not found")
    print("Download file for reading...")
    os.system('curl -O http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2015/2015_bronx.xls')
    os.system('mv 2015_bronx.xls ' + path)

if os.path.isfile(path + '/2015_bronx.xls'):
    print("File in place, read file")
    bx2015 = pd.read_excel(path + '/2015_bronx.xls', skiprows=4)

File not found
Download file for reading...
File in place, read file


In [13]:
bx2015.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,2905,28,,A9,1665 WASHINGTON AVENUE,,...,1,0,1,4750,2619,1899,1,A9,0,2015-08-07
1,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,2905,28,,A9,1665 WASHINGTON AVENUE,,...,1,0,1,4750,2619,1899,1,A9,0,2015-08-07
2,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3035,2,,S1,441 EAST 178 STREET,,...,1,1,2,1287,2528,1899,1,S1,0,2015-12-18
3,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3039,28,,A1,2329 WASHINGTON AVE,,...,1,0,1,1103,1290,1910,1,A1,300000,2015-02-20
4,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3039,36,,S1,455 EAST 184 STREET,,...,1,1,2,2500,2452,1910,1,S1,0,2015-08-18


In [16]:
if (not os.path.isfile(path + '/2015_brooklyn.xls')):
    print("File not found")
    print("Download file for reading...")
    os.system('curl -O http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2015/2015_brooklyn.xls')
    os.system('mv 2015_brooklyn.xls ' + path)

if os.path.isfile(path + '/2015_brooklyn.xls'):
    print("File in place, read file")
    bk2015 = pd.read_excel(path + '/2015_brooklyn.xls', skiprows=4)

File not found
Download file for reading...
File in place, read file


In [17]:
if (not os.path.isfile(path + '/2015_queens.xls')):
    print("File not found")
    print("Download file for reading...")
    os.system('curl -O http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2015/2015_queens.xls')
    os.system('mv 2015_queens.xls ' + path)

if os.path.isfile(path + '/2015_queens.xls'):
    print("File in place, read file")
    qn2015 = pd.read_excel(path + '/2015_queens.xls', skiprows=4)

File not found
Download file for reading...
File in place, read file


In [18]:
if (not os.path.isfile(path + '/2015_statenisland.xls')):
    print("File not found")
    print("Download file for reading...")
    os.system('curl -O http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2015/2015_statenisland.xls')
    os.system('mv 2015_statenisland.xls ' + path)

if os.path.isfile(path + '/2015_statenisland.xls'):
    print("File in place, read file")
    si2015 = pd.read_excel(path + '/2015_statenisland.xls', skiprows=4)

File not found
Download file for reading...
File in place, read file


In [6]:
def fixColumns(df):
    cols = df.columns.str.replace('\n', '').str.replace(' ', '_')
    df.columns = cols

In [7]:
def showDesc(df):
    print('Number of sales with $0 price: {}'.format(df[df['SALE_PRICE'] == 0]['LOT'].count()))
    print('Number of sales with negative price: {}'.format(df[df['SALE_PRICE'] < 0]['LOT'].count()))
    print('Number of sales with 0 year built: {}'.format(df[df['YEAR_BUILT'] == 0]['LOT'].count()))

In [9]:
fixColumns(mn2015)

In [11]:
showDesc(mn2015)

Number of sales with $0 price: 5461
Number of sales with negative price: 0
Number of sales with 0 year built: 2636


In [14]:
fixColumns(bx2015)

In [15]:
showDesc(bx2015)

Number of sales with $0 price: 1880
Number of sales with negative price: 0
Number of sales with 0 year built: 758


In [19]:
fixColumns(bk2015)

In [20]:
showDesc(bk2015)

Number of sales with $0 price: 9190
Number of sales with negative price: 0
Number of sales with 0 year built: 2162


In [21]:
fixColumns(qn2015)

In [22]:
showDesc(qn2015)

Number of sales with $0 price: 8059
Number of sales with negative price: 0
Number of sales with 0 year built: 1249


In [23]:
fixColumns(si2015)

In [24]:
showDesc(si2015)

Number of sales with $0 price: 2464
Number of sales with negative price: 0
Number of sales with 0 year built: 426
